# Auotomated Mail Sender

## A program that send out top 10 rising posts of reddit through mail, every hour

1. Type of posts can be customised
2. Time interval can be customised as well

In [ ]:
# Imporing necessary modules
import schedule
import time

# Defining a function, that runs over and over again.
def mailsender():
    import pandas as pd
    import numpy as np
    import praw
    import yagmail
    
    # Scraping reddit data
    reddit = praw.Reddit(client_id='<client_id>', 
                     client_secret='<secret_key>', 
                     user_agent='scraper_101', 
                     username='<reddit_user_name>', 
                     password='<password>')
    
    no_subreddit = reddit.subreddit('all')
    new = no_subreddit.rising(limit=10000)
    
    dict =        { "title":[],
                "subreddit":[],
                "Upvotes":[],
                "Upvote_ratio":[],
                "id":[], 
                "url":[], 
                "comms_num": [], 
                "created": [], 
                "body":[],
                "NSFW(Not safe for work)":[] }
    
    # Organizing data to a dataframe
    for submission in new:
        dict["title"].append(submission.title)
        dict['subreddit'].append(submission.subreddit)
        dict["Upvotes"].append(submission.score)
        dict["id"].append(submission.id)
        dict["url"].append(submission.url)
        dict['Upvote_ratio'].append(submission.upvote_ratio)
        dict["comms_num"].append(submission.num_comments)
        dict["created"].append(submission.created)
        dict["body"].append(submission.selftext)
        dict["NSFW(Not safe for work)"].append(submission.over_18)
        
    df = pd.DataFrame(dict)
    df.drop_duplicates(subset=['id'], inplace=True)
    
    # Cleaning data
    import datetime as dt
    def get_date(created):
        return dt.datetime.fromtimestamp(created)
    x=[]
    for i in df['created']:
        x.append(get_date(i))
    df['DateTime']=x
    df.drop(['created'],axis=1,inplace=True)
    
    # Sorting required top 10 data, considering below metrics.    
    df.sort_values(['Upvotes','Upvote_ratio','DateTime'],ascending=False,inplace=True)
    df_top10= df.head(10)
    df_mail=df_top10.drop(['id','comms_num','NSFW(Not safe for work)','Upvotes','Upvote_ratio','body'],axis=1)
    
    # Integrating this to your mail    
    user=yagmail.SMTP(user='<mail_id>',password='<password>')
    print(user.send(to=['list of mail_IDs'], \
             subject='First automated test email', \
             contents=['Top 10 rising reddit posts that you might like','\n',df_mail]))
    
    # Scheduling a timer to time your mails, every 1 hour this case
    schedule.every(60).minutes.do(mailsender)
    while True:
        schedule.run_pending()
        time.sleep(60)
        
print(mailsender())